# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [26]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [27]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [28]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'epa', 'gardenDB', 'grovedb', 'local', 'petsitly_marketing', 'uk_food']


In [29]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [30]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [31]:
# assign the collection to a variable
est = db.establishments


## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [32]:
# Create a dictionary for the new restaurant data
rest_data = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance": 4623.9723280747176,
    "NewRatingPending":True
}

In [33]:
# Insert the new restaurant into the collection
est.insert_one(rest_data)

In [ ]:
# Check that the new restaurant was inserted
new = est.find_one(rest_data)

for key in new:
    pprint(new)

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Find the BusinessTypeID for "Restauranvt/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
bus_id_data = est.find({"BusinessType": "Restaurant/Cafe/Canteen"})
for bus_id in bus_id_data:
    print(bus_id["BusinessTypeID"], bus_id["BusinessType"])

3. Update the new restaurant with the `BusinessTypeID` you found.

In [36]:
# Update the new restaurant with the correct BusinessTypeID
filter = {"BusinessType": "Restaurant/Cafe/Canteen"}
est.update_one(filter, {"$set": { "BusinessTypeID": 1 }})

In [ ]:
# Confirm that the new restaurant was updated
new = est.find(filter)
for key in new:
    pprint(key)

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [38]:
# Find how many documents have LocalAuthorityName as "Dover"
count = est.count_documents({"LocalAuthorityName": "Dover"})
print(count)

994


In [39]:
# Delete all documents where LocalAuthorityName is "Dover"
est.delete_many({"LocalAuthorityName": "Dover"})

In [40]:
# Check if any remaining documents include Dover
dover_data = est.find({"LocalAuthorityName": "Dover"})
for key in dover_data:
    pprint(key)

In [ ]:
# Check that other documents remain with 'find_one'
other_data = est.find_one()
for key in other_data:
    pprint(key)

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [42]:
# Change the data type from String to Decimal for longitude and latitude
est.update_many({}, [{'$set': {'geocode.longitude': {'$toDouble': '$geocode.longitude'}}}])
est.update_many({}, [{'$set': {'geocode.latitude': {'$toDouble': '$geocode.latitude'}}}])
# FIX THIS CODE!!! 

Use `update_many` to convert `RatingValue` to integer numbers.

In [43]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
est.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [44]:
# Change the data type from String to Integer for RatingValue
est.update_many({}, [{'$set': {'RatingValue': {'$toInt': '$RatingValue'}}}])

In [ ]:
# Check that the coordinates and rating value are now numbers
data = est.find()
for key in data:
    pprint(key)